Загрузите файл со статьями статей из Википедии. Каждая строка в файле имеет следующий формат:

URL статьи \<tab> название статьи \<tab> текст 


Напишите с помощью PySpark или Ray:

1) Напишите программу, которая находит самое длинное слово.
2) Напишите программу, которая находит среднюю длину слов.
3) Напишите программу, которая находит самое частоупотребляемое слово, состоящее из латинских букв.
4) Все слова, которые более чем в половине случаев начинаются с большой буквы и встречаются больше 10 раз.
5) Напишите программу, которая с помощью статистики определяет устойчивые сокращения вида пр., др., ...
6) Напишите программу, которая с помощью статистики определяет устойчивые сокращения вида т.п., н.э., ...
7) Напишите программу, которая с помощью статистики находит имена, употребляющиеся в статьях.

Из подзаданий 5-7 достаточно сделать любые два.

# Загрузка данных

In [207]:
!pip3 install findspark



[notice] A new release of pip is available: 23.2.1 -> 24.1.1
[notice] To update, run: pip3 install --upgrade pip


In [1]:
import findspark
findspark.init()

In [2]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[1]").appName("SparkByExamples.com").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/07/03 11:05:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [47]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

schema = StructType([
    StructField("text", StringType(), True)])
    
data = spark.\
    read.\
    option("delimiter", "\n").\
    option("encoding", "utf-8").\
    csv('wiki.txt', schema=schema)

data.take(1)
# data.select(split(data.s, '[\t]', -1).alias('s')).collect()

[Row(text='http://ru.wikipedia.org/wiki?curid=7\tЛитва\tЛитва  Литва́ (), официальное название\xa0— Лито́вская Респу́блика ()\xa0— государство, географически расположенное в Северной Европе, по культурным критериям её относят к Центральной Европе. Столица Литвы\xa0— Вильнюс. Расположена на восточном побережье Балтийского моря. На севере граничит с Латвией, на юго-востоке\xa0— с Белоруссией, на юго-западе\xa0— c Польшей и Калининградской областью России. Член ООН с 1991 года, ЕС и НАТО\xa0— с 2004 года. Входит в Шенгенскую зону. Поверхность\xa0— равнинная со следами древнего оледенения. Поля и луга занимают 57\xa0% территории, леса и кустарники\xa0— 30\xa0%, болота\xa0— 3\xa0%, внутренние воды\xa0— 4\xa0%. Высшая точка\xa0— 293,84 м над уровнем моря\xa0— холм Аукштояс () (или Аукштасис калнас ()) в юго-восточной части страны, в 23,5\xa0км от Вильнюса. Более 3 тыс. озёр (1,5\xa0% территории): крупнейшее из них\xa0— Друкшяй (, ) на границе Латвии, Литвы и Белоруссии (площадь 44,8 км²), са

# Data preprocessing

In [48]:
from pyspark.sql.functions import *
data = data.withColumn('text', regexp_replace('text', '\xa0', ' '))

data.take(1)

[Row(text='http://ru.wikipedia.org/wiki?curid=7\tЛитва\tЛитва  Литва́ (), официальное название — Лито́вская Респу́блика () — государство, географически расположенное в Северной Европе, по культурным критериям её относят к Центральной Европе. Столица Литвы — Вильнюс. Расположена на восточном побережье Балтийского моря. На севере граничит с Латвией, на юго-востоке — с Белоруссией, на юго-западе — c Польшей и Калининградской областью России. Член ООН с 1991 года, ЕС и НАТО — с 2004 года. Входит в Шенгенскую зону. Поверхность — равнинная со следами древнего оледенения. Поля и луга занимают 57 % территории, леса и кустарники — 30 %, болота — 3 %, внутренние воды — 4 %. Высшая точка — 293,84 м над уровнем моря — холм Аукштояс () (или Аукштасис калнас ()) в юго-восточной части страны, в 23,5 км от Вильнюса. Более 3 тыс. озёр (1,5 % территории): крупнейшее из них — Друкшяй (, ) на границе Латвии, Литвы и Белоруссии (площадь 44,8 км²), самое глубокое — Таурагнас (, 61 м), самое длинное — Асвея 

In [49]:
# removing url and header
from pyspark.sql.functions import col, udf
from pyspark.sql.types import StringType


get_text_udf = udf(lambda x:" ".join(x.split('\t')[2:]),StringType()) 
data = data.withColumn("text", get_text_udf(col("text")))

data.take(1)

[Row(text='Литва  Литва́ (), официальное название — Лито́вская Респу́блика () — государство, географически расположенное в Северной Европе, по культурным критериям её относят к Центральной Европе. Столица Литвы — Вильнюс. Расположена на восточном побережье Балтийского моря. На севере граничит с Латвией, на юго-востоке — с Белоруссией, на юго-западе — c Польшей и Калининградской областью России. Член ООН с 1991 года, ЕС и НАТО — с 2004 года. Входит в Шенгенскую зону. Поверхность — равнинная со следами древнего оледенения. Поля и луга занимают 57 % территории, леса и кустарники — 30 %, болота — 3 %, внутренние воды — 4 %. Высшая точка — 293,84 м над уровнем моря — холм Аукштояс () (или Аукштасис калнас ()) в юго-восточной части страны, в 23,5 км от Вильнюса. Более 3 тыс. озёр (1,5 % территории): крупнейшее из них — Друкшяй (, ) на границе Латвии, Литвы и Белоруссии (площадь 44,8 км²), самое глубокое — Таурагнас (, 61 м), самое длинное — Асвея (, длина 30 км) у местечка Дубингяй. Климат п

In [51]:
# Replace stressed signs

replacer = { "а́" : "а", "е́" : "е", "о́" : "о", "у́" : "у",
      "я́" : "я", "ю́" : "ю", "ы́" : "ы", "и́" : "и",
      "ё́" : "ё", "А́" : "А", "Е́" : "Е", "О́" : "О",
      "У́" : "У", "Я́" : "Я", "Ю́" : "Ю", "Ы́" : "Ы",
      "И́" : "И", "Э́" : "Э", "э́" : "э"}

for letter in repalcer:
    data = data.select(regexp_replace(col("text"), letter, replacer[letter]).alias('text'))

data.take(1)

[Row(text='Литва  Литва (), официальное название — Литовская Республика () — государство, географически расположенное в Северной Европе, по культурным критериям её относят к Центральной Европе. Столица Литвы — Вильнюс. Расположена на восточном побережье Балтийского моря. На севере граничит с Латвией, на юго-востоке — с Белоруссией, на юго-западе — c Польшей и Калининградской областью России. Член ООН с 1991 года, ЕС и НАТО — с 2004 года. Входит в Шенгенскую зону. Поверхность — равнинная со следами древнего оледенения. Поля и луга занимают 57 % территории, леса и кустарники — 30 %, болота — 3 %, внутренние воды — 4 %. Высшая точка — 293,84 м над уровнем моря — холм Аукштояс () (или Аукштасис калнас ()) в юго-восточной части страны, в 23,5 км от Вильнюса. Более 3 тыс. озёр (1,5 % территории): крупнейшее из них — Друкшяй (, ) на границе Латвии, Литвы и Белоруссии (площадь 44,8 км²), самое глубокое — Таурагнас (, 61 м), самое длинное — Асвея (, длина 30 км) у местечка Дубингяй. Климат пере

In [26]:
# clean_text_udf = udf(lambda x:" ".join(c for c in x if c.isalpha() or c == " "),StringType()) 
# data = data.withColumn("text", clean_text_udf(col("text")))
# data.take(1)

[Row(text='Литва  Литва  официальное название  Литовская Республика   государство географически расположенное в Северной Европе по культурным критериям её относят к Центральной Европе Столица Литвы  Вильнюс Расположена на восточном побережье Балтийского моря На севере граничит с Латвией на юговостоке  с Белоруссией на югозападе  c Польшей и Калининградской областью России Член ООН с  года ЕС и НАТО  с  года Входит в Шенгенскую зону Поверхность  равнинная со следами древнего оледенения Поля и луга занимают   территории леса и кустарники    болота    внутренние воды    Высшая точка   м над уровнем моря  холм Аукштояс  или Аукштасис калнас  в юговосточной части страны в  км от Вильнюса Более  тыс озёр   территории крупнейшее из них  Друкшяй   на границе Латвии Литвы и Белоруссии площадь  км самое глубокое  Таурагнас   м самое длинное  Асвея  длина  км у местечка Дубингяй Климат переходный от морского к континентальному Средняя температура зимой минус  С летом  С Выпадает  мм осадков в год

In [52]:
data = data.select(regexp_replace(col("text"), r'http\S+', "").alias('text'))
data.take(1)

[Row(text='Литва  Литва (), официальное название — Литовская Республика () — государство, географически расположенное в Северной Европе, по культурным критериям её относят к Центральной Европе. Столица Литвы — Вильнюс. Расположена на восточном побережье Балтийского моря. На севере граничит с Латвией, на юго-востоке — с Белоруссией, на юго-западе — c Польшей и Калининградской областью России. Член ООН с 1991 года, ЕС и НАТО — с 2004 года. Входит в Шенгенскую зону. Поверхность — равнинная со следами древнего оледенения. Поля и луга занимают 57 % территории, леса и кустарники — 30 %, болота — 3 %, внутренние воды — 4 %. Высшая точка — 293,84 м над уровнем моря — холм Аукштояс () (или Аукштасис калнас ()) в юго-восточной части страны, в 23,5 км от Вильнюса. Более 3 тыс. озёр (1,5 % территории): крупнейшее из них — Друкшяй (, ) на границе Латвии, Литвы и Белоруссии (площадь 44,8 км²), самое глубокое — Таурагнас (, 61 м), самое длинное — Асвея (, длина 30 км) у местечка Дубингяй. Климат пере

In [53]:
data = data.select(regexp_replace(col("text"), r'[^A-Za-zА-Яа-я]', " ").alias('text'))
data.take(1)

[Row(text='Литва  Литва     официальное название   Литовская Республика      государство  географически расположенное в Северной Европе  по культурным критериям е  относят к Центральной Европе  Столица Литвы   Вильнюс  Расположена на восточном побережье Балтийского моря  На севере граничит с Латвией  на юго востоке   с Белоруссией  на юго западе   c Польшей и Калининградской областью России  Член ООН с      года  ЕС и НАТО   с      года  Входит в Шенгенскую зону  Поверхность   равнинная со следами древнего оледенения  Поля и луга занимают      территории  леса и кустарники         болота        внутренние воды        Высшая точка          м над уровнем моря   холм Аукштояс     или Аукштасис калнас     в юго восточной части страны  в      км от Вильнюса  Более   тыс  оз р        территории   крупнейшее из них   Друкшяй      на границе Латвии  Литвы и Белоруссии  площадь      км    самое глубокое   Таурагнас       м   самое длинное   Асвея    длина    км  у местечка Дубингяй  Климат пере

In [54]:
data = data.select(regexp_replace(col("text"), " +", " ").alias('text'))
data.take(1)


[Row(text='Литва Литва официальное название Литовская Республика государство географически расположенное в Северной Европе по культурным критериям е относят к Центральной Европе Столица Литвы Вильнюс Расположена на восточном побережье Балтийского моря На севере граничит с Латвией на юго востоке с Белоруссией на юго западе c Польшей и Калининградской областью России Член ООН с года ЕС и НАТО с года Входит в Шенгенскую зону Поверхность равнинная со следами древнего оледенения Поля и луга занимают территории леса и кустарники болота внутренние воды Высшая точка м над уровнем моря холм Аукштояс или Аукштасис калнас в юго восточной части страны в км от Вильнюса Более тыс оз р территории крупнейшее из них Друкшяй на границе Латвии Литвы и Белоруссии площадь км самое глубокое Таурагнас м самое длинное Асвея длина км у местечка Дубингяй Климат переходный от морского к континентальному Средняя температура зимой минус С летом С Выпадает мм осадков в год Название Литва впервые упомянуто в Кведлин

In [55]:
count_rdd = data.select("text").rdd.flatMap(lambda x: x[0].split(' ')) \
              .map(lambda x: (x, 1)).reduceByKey(lambda x,y: x+y)



In [56]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

counter_schema = StructType([
    StructField("word", StringType(), True),
    StructField("count", IntegerType(), True)])

counter = spark.createDataFrame(count_rdd, schema=counter_schema)

counter.take(1)

[Row(word='Литва', count=85)]

1) Напишите программу, которая находит самое длинное слово.

In [57]:
counter = counter.withColumn('word_length', length(counter.word))
counter.take(1)

[Row(word='Литва', count=85, word_length=5)]

In [58]:
max_len = counter.select(max(counter.word_length).alias('max_len')).collect()[0]['max_len']

In [59]:
counter.filter(counter.word_length == max_len).take(1)

[Row(word='dreihundertvierundsechzigtausendachthundertneunzehn', count=1, word_length=51)]

2) Напишите программу, которая находит среднюю длину слов.


In [60]:
counter.select(mean(counter.word_length)).collect()

[Row(avg(word_length)=9.057915954540732)]

3) Напишите программу, которая находит самое частоупотребляемое слово, состоящее из латинских букв.


In [61]:
english_word_regex = "^[a-zA-Z]+$"

english_words = counter.filter(regexp_extract(col("word"), english_word_regex, 0) != "")

english_words.take(4)

[Row(word='c', count=775, word_length=1),
 Row(word='X', count=1423, word_length=1),
 Row(word='XIII', count=944, word_length=4),
 Row(word='XIV', count=849, word_length=3)]

In [62]:
max_freq = english_words.select(max(english_words['count']).alias('max_freq')).collect()[0]['max_freq']

In [63]:
english_words.filter(english_words['count'] == max_freq).take(1)

[Row(word='formula', count=11589, word_length=7)]

4) Все слова, которые более чем в половине случаев начинаются с большой буквы и встречаются больше 10 раз.


Идея:
- Разделить слова на две таблицы: с заглавными буквами и с прописными
- В таблице с заглавными буквами сделать все слова прописными
- При помощи join посчитать долю заглавных относительно всех
- Если больше 50% и встречается больше 10 раз - вывести слово

In [64]:
capital_regex = "^([А-ЯA-Z][а-яa-z]+)+$"

big_letters = counter.filter(regexp_extract(col("word"), capital_regex, 0) != "")
big_letters.take(10)

[Row(word='Литва', count=85, word_length=5),
 Row(word='Литовская', count=19, word_length=9),
 Row(word='Республика', count=696, word_length=10),
 Row(word='Северной', count=1004, word_length=8),
 Row(word='Европе', count=1509, word_length=6),
 Row(word='Центральной', count=488, word_length=11),
 Row(word='Столица', count=176, word_length=7),
 Row(word='Литвы', count=251, word_length=5),
 Row(word='Вильнюс', count=46, word_length=7),
 Row(word='Расположена', count=29, word_length=11)]

In [70]:
big_letters = big_letters.withColumn('word', lower(col('word')))

big_letters.take(10)



[Row(word='литва', count=85, word_length=5),
 Row(word='литовская', count=19, word_length=9),
 Row(word='республика', count=696, word_length=10),
 Row(word='северной', count=1004, word_length=8),
 Row(word='европе', count=1509, word_length=6),
 Row(word='центральной', count=488, word_length=11),
 Row(word='столица', count=176, word_length=7),
 Row(word='литвы', count=251, word_length=5),
 Row(word='вильнюс', count=46, word_length=7),
 Row(word='расположена', count=29, word_length=11)]

In [71]:
small_regex = "^([а-яa-z]+)+$"

small_letters = counter.filter(regexp_extract(col("word"), small_regex, 0) != "")
small_letters.take(10)

[Row(word='официальное', count=231, word_length=11),
 Row(word='название', count=3710, word_length=8),
 Row(word='государство', count=1201, word_length=11),
 Row(word='географически', count=30, word_length=13),
 Row(word='расположенное', count=68, word_length=13),
 Row(word='в', count=322385, word_length=1),
 Row(word='по', count=49294, word_length=2),
 Row(word='культурным', count=118, word_length=10),
 Row(word='критериям', count=47, word_length=9),
 Row(word='е', count=12220, word_length=1)]

In [80]:
big_letters = big_letters.withColumnRenamed("word", "capital_word")\
           .withColumnRenamed("count", "capital_count")\
           .withColumnRenamed("word_length", "capital_word_length")

In [83]:
joined_df = small_letters.join(big_letters, small_letters.word == big_letters.capital_word, "outer",)

joined_df.take(1)

[Row(word='a', count=900, word_length=1, capital_word=None, capital_count=None, capital_word_length=None)]

In [87]:
paired_words = joined_df.where(col("word").isNotNull() & col("capital_word").isNotNull())
paired_words.take(3)

[Row(word='температура', count=1477, word_length=11, capital_word='температура', capital_count=220, capital_word_length=11),
 Row(word='вассалами', count=31, word_length=9, capital_word='вассалами', capital_count=2, capital_word_length=9),
 Row(word='абвер', count=9, word_length=5, capital_word='абвер', capital_count=10, capital_word_length=5)]

In [93]:
paired_words.where((col('capital_count') / col('count') > 1) & (col('capital_count') > 10)).take(10)

[Row(word='всесоюзном', count=10, word_length=10, capital_word='всесоюзном', capital_count=22, capital_word_length=10),
 Row(word='вяземский', count=1, word_length=9, capital_word='вяземский', capital_count=14, capital_word_length=9),
 Row(word='бор', count=32, word_length=3, capital_word='бор', capital_count=117, capital_word_length=3),
 Row(word='киевской', count=22, word_length=8, capital_word='киевской', capital_count=173, capital_word_length=8),
 Row(word='ганзейского', count=3, word_length=11, capital_word='ганзейского', capital_count=15, capital_word_length=11),
 Row(word='telnet', count=11, word_length=6, capital_word='telnet', capital_count=21, capital_word_length=6),
 Row(word='art', count=11, word_length=3, capital_word='art', capital_count=24, capital_word_length=3),
 Row(word='тувалу', count=14, word_length=6, capital_word='тувалу', capital_count=134, capital_word_length=6),
 Row(word='мюнхенский', count=1, word_length=10, capital_word='мюнхенский', capital_count=12, capit

In [92]:
paired_words.count()

51477